In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Population-Time-RNN'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data/POPH.csv')

In [3]:
data

   realtime_start      value        date realtime_end
0      2019-12-06   76094000  1900-01-01   2019-12-06
1      2019-12-06   77584000  1901-01-01   2019-12-06
2      2019-12-06   79163000  1902-01-01   2019-12-06
3      2019-12-06   80632000  1903-01-01   2019-12-06
4      2019-12-06   82166000  1904-01-01   2019-12-06
..            ...        ...         ...          ...
95     2019-12-06  262803276  1995-01-01   2019-12-06
96     2019-12-06  265228572  1996-01-01   2019-12-06
97     2019-12-06  267783607  1997-01-01   2019-12-06
98     2019-12-06  270248003  1998-01-01   2019-12-06
99     2019-12-06  272690813  1999-01-01   2019-12-06

[100 rows x 4 columns]

In [4]:
data = torch.from_numpy(np.array(data['value'].tolist()))

In [5]:
data

tensor([ 76094000,  77584000,  79163000,  80632000,  82166000,  83822000,
         85450000,  87008000,  88710000,  90490000,  92407000,  93863000,
         95335000,  97225000,  99111000, 100546000, 101961000, 103268000,
        103208000, 104514000, 106461000, 108538000, 110049000, 111947000,
        114109000, 115829000, 117397000, 119035000, 120509000, 121767000,
        123076741, 124039648, 124840471, 125578763, 126373773, 127250232,
        128053180, 128824829, 129824939, 130879718, 132122446, 133402471,
        134859553, 136739353, 138397345, 139928165, 141388566, 144126071,
        146631302, 149188130, 152271417, 154877889, 157552740, 160184192,
        163025854, 165931202, 168903031, 171984130, 174881904, 177829628,
        180671158, 183691481, 186537737, 189241798, 191888791, 194302963,
        196560338, 198712056, 200706052, 202676946, 205052174, 207660677,
        209896021, 211908788, 213853928, 215973199, 218035164, 220239425,
        222584545, 225055487, 22722468

In [6]:
len(data)

100

In [7]:
data

tensor([ 76094000,  77584000,  79163000,  80632000,  82166000,  83822000,
         85450000,  87008000,  88710000,  90490000,  92407000,  93863000,
         95335000,  97225000,  99111000, 100546000, 101961000, 103268000,
        103208000, 104514000, 106461000, 108538000, 110049000, 111947000,
        114109000, 115829000, 117397000, 119035000, 120509000, 121767000,
        123076741, 124039648, 124840471, 125578763, 126373773, 127250232,
        128053180, 128824829, 129824939, 130879718, 132122446, 133402471,
        134859553, 136739353, 138397345, 139928165, 141388566, 144126071,
        146631302, 149188130, 152271417, 154877889, 157552740, 160184192,
        163025854, 165931202, 168903031, 171984130, 174881904, 177829628,
        180671158, 183691481, 186537737, 189241798, 191888791, 194302963,
        196560338, 198712056, 200706052, 202676946, 205052174, 207660677,
        209896021, 211908788, 213853928, 215973199, 218035164, 220239425,
        222584545, 225055487, 22722468

In [8]:
data = torch.from_numpy(np.array(data['value'].tolist())).view(1,-1)

In [9]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Population-Time-RNN'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [10]:
data = pd.read_csv('./data/POPH.csv')

In [11]:
data = torch.from_numpy(np.array(data['value'].tolist())).view(1,-1)

In [12]:
data

tensor([[ 76094000,  77584000,  79163000,  80632000,  82166000,  83822000,
          85450000,  87008000,  88710000,  90490000,  92407000,  93863000,
          95335000,  97225000,  99111000, 100546000, 101961000, 103268000,
         103208000, 104514000, 106461000, 108538000, 110049000, 111947000,
         114109000, 115829000, 117397000, 119035000, 120509000, 121767000,
         123076741, 124039648, 124840471, 125578763, 126373773, 127250232,
         128053180, 128824829, 129824939, 130879718, 132122446, 133402471,
         134859553, 136739353, 138397345, 139928165, 141388566, 144126071,
         146631302, 149188130, 152271417, 154877889, 157552740, 160184192,
         163025854, 165931202, 168903031, 171984130, 174881904, 177829628,
         180671158, 183691481, 186537737, 189241798, 191888791, 194302963,
         196560338, 198712056, 200706052, 202676946, 205052174, 207660677,
         209896021, 211908788, 213853928, 215973199, 218035164, 220239425,
         222584545, 22505

In [13]:
data_input = data[:1,:-1]
data_target = data[:1,1:]

In [14]:
data_input

tensor([[ 76094000,  77584000,  79163000,  80632000,  82166000,  83822000,
          85450000,  87008000,  88710000,  90490000,  92407000,  93863000,
          95335000,  97225000,  99111000, 100546000, 101961000, 103268000,
         103208000, 104514000, 106461000, 108538000, 110049000, 111947000,
         114109000, 115829000, 117397000, 119035000, 120509000, 121767000,
         123076741, 124039648, 124840471, 125578763, 126373773, 127250232,
         128053180, 128824829, 129824939, 130879718, 132122446, 133402471,
         134859553, 136739353, 138397345, 139928165, 141388566, 144126071,
         146631302, 149188130, 152271417, 154877889, 157552740, 160184192,
         163025854, 165931202, 168903031, 171984130, 174881904, 177829628,
         180671158, 183691481, 186537737, 189241798, 191888791, 194302963,
         196560338, 198712056, 200706052, 202676946, 205052174, 207660677,
         209896021, 211908788, 213853928, 215973199, 218035164, 220239425,
         222584545, 22505

In [15]:
data_targetA

In [16]:
data_target

tensor([[ 77584000,  79163000,  80632000,  82166000,  83822000,  85450000,
          87008000,  88710000,  90490000,  92407000,  93863000,  95335000,
          97225000,  99111000, 100546000, 101961000, 103268000, 103208000,
         104514000, 106461000, 108538000, 110049000, 111947000, 114109000,
         115829000, 117397000, 119035000, 120509000, 121767000, 123076741,
         124039648, 124840471, 125578763, 126373773, 127250232, 128053180,
         128824829, 129824939, 130879718, 132122446, 133402471, 134859553,
         136739353, 138397345, 139928165, 141388566, 144126071, 146631302,
         149188130, 152271417, 154877889, 157552740, 160184192, 163025854,
         165931202, 168903031, 171984130, 174881904, 177829628, 180671158,
         183691481, 186537737, 189241798, 191888791, 194302963, 196560338,
         198712056, 200706052, 202676946, 205052174, 207660677, 209896021,
         211908788, 213853928, 215973199, 218035164, 220239425, 222584545,
         225055487, 22722

In [17]:
data_input = data[:1,:-1].float().to(device)
data_target = data[:1,1:].float().to(device)

In [18]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)

In [19]:
Model()

In [20]:
model = Model(128).to(device)

In [21]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [22]:
criterion = MSELoss()

In [23]:
epochs = 100

In [24]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        preds = preds.to(device)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].cpu().detach().numpy()
        n = data_input.shape[1]
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.watch(model)
wandb.finish()

In [25]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [26]:
model = Model(128).to(device)

In [27]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [28]:
criterion = MSELoss()

In [29]:
epochs = 100

In [30]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        preds = preds.to(device)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].cpu().detach().numpy()
        n = data_input.shape[1]
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.watch(model)
wandb.finish()